In [5]:
import torch
from torch import nn


In [6]:
with open("input.txt", 'r', encoding="UTF-8") as f:
    text = f.read()
    chars = sorted(set(text))
    vocab_size = len(chars)


stoi = {char:integer for integer, char in enumerate(chars)}
itos = {integer:char for integer, char in enumerate(chars)}

encode = lambda enc: [stoi[c] for c in enc]
decode = lambda dec: "".join([itos[i] for i in dec])

print(encode("\n"))
print(decode(encode("\n")))

vocab_size = len(stoi)

print(vocab_size)

[0]


66


In [7]:
inputs = []
labels = []
context_size = 3
for n in names[:]:
    context = [0] * context_size
    for char in n + ".":
        idx = stoi[char]
        inputs.append(context)
        labels.append(idx)
        context = context[1:] + [idx]

inputs = torch.tensor(inputs)
labels = torch.tensor(labels)

print(inputs)
print(labels)

NameError: name 'names' is not defined

In [8]:
class MLP(nn.Module):
    pass

In [9]:
n_embd = 2

C = torch.randn((vocab_size, n_embd))

In [10]:
embtest = C[inputs]
print(vocab_size)
print(inputs.shape)
print(embtest.shape)

embtest = embtest.view(-1, context_size*n_embd)
print(embtest.shape)

66


AttributeError: 'list' object has no attribute 'shape'

In [29]:
class MLP(nn.Module):
    def __init__(self, vocab_size, n_embd):
        super(MLP, self).__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        self.positional_emb_table = nn.Embedding(context_size, n_embd)
        self.lm_head = nn.Linear(in_features=n_embd, out_features=vocab_size)

        self.query = nn.Linear(in_features=n_embd, out_features=n_embd)
        self.key = nn.Linear(in_features=n_embd, out_features=n_embd)

    def forward(self, x, batched:bool):
        tok_emb = self.token_embedding(x)
        #print(f"x.shape[1] --> {x.shape[1]}")
        pos_emb = self.positional_emb_table(torch.arange(x.shape[1])) # --> context_size X n_embd
        #print(pos_emb.shape)
        result_emb = tok_emb + pos_emb

        Q = self.query(result_emb)
        K = self.key(result_emb)

        print(Q.shape)
        print(K.shape)

        #dot_simillarity = Q @ K
        #print(dot_simillarity.shape)
        #print(dot_simillarity)



        #test = self.token_embedding(torch.tensor([0])).shape --> [2] #basically it's a 1 X 1 X 2 but squeezed
        #test = self.token_embedding(torch.tensor([0, 2])).shape --> [2, 2]
        #test = self.token_embedding(torch.tensor([0, 2, 2])).shape --> [3, 2] #the n_embd is always preserved so it can go into the lm_head
        #print(test)
        #emb = emb.view(-1, context_size*n_embd)
        #OR --> B, T, C = emb.shape #batch --> how many inputs in the batch #Time --> context size #Channels --> n_embd
        logits = self.lm_head(result_emb)
        if batched:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) #96 X 2 #assuming batch_size = 32 and n_embd = 2

        #The explanation below is for a batch size of 32 and vocab size of 27
        #logits = self.lm_head(emb) # turns this into --> 96 X 27 #so there must be 96 labels in total to be compared those 96 inputs #because of B * T in the view above it's not 32 inputs of 3 characters anymore, it's now 96 characters with 27 probabilities each
        #if the input is not bached, then there is no batch input dimension nor time, the only output plucked out of the emb table are 2 channels, the shape of it being [2] ex --> [-2.243, 0.763]


        return logits

    def generator(self, starting_idx, max_new_tokens):

        for _ in range(max_new_tokens):
            #max_new_tokens is needed since there is no special character to stop the generation
            logits = self(starting_idx, False) #plucks out the 0th row of the token embedding table
            #logits are of shape --> B*T X vocab_size or just batch_size * n_context_character X vocab_size

            #then based on the last character, predict the next one

            #print(logits.shape) #since it's being passed as batched, there is no batch_dimension in the logits
            #but if it's passed as unbatched, there will be an extra dimension at the beggining of logits
            #print(logits[:, -1, :]) # --> no need for the last : but just in case it can be useful
            #if it's passed as unbatched, there will be an extra dimension at the beggining, thus the need to do it like this --> [:, -1, :]
            probs = torch.softmax(logits[:, -1, :], dim=1) #no dim passed because is unbatched, if batched, pass dim as 1
            new_idx = torch.multinomial(probs, num_samples=1)
            starting_idx = torch.cat((starting_idx, new_idx), dim=1)
            #print(starting_idx)
        return starting_idx



In [26]:
def get_batch(data, batch_size, context_size):
    batch_idx = torch.randint(0, data.shape[0]-context_size, (batch_size,))
    x = torch.stack([data[idx:idx+context_size] for idx in batch_idx])
    y = torch.stack([data[idx+1:idx+1+context_size] for idx in batch_idx])

    #decode x to view what it is
    #print(x[0].tolist())
    #print(decode(x[0].tolist()))
    #print(decode(y[0].tolist()))

    return (x, y)

data = torch.tensor(encode(text))
x, y = get_batch(data, 32, 3)
print(y.shape)

torch.Size([32, 3])


In [27]:
n_embd = 32
context_size = 256 #also max context for predictions
batch_size = 64

model = MLP(vocab_size=vocab_size, n_embd=n_embd)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

In [28]:
epochs = 1

for epoch in tqdm(range(epochs)):
    x, y = get_batch(data, batch_size, context_size)
    #print(x.shape)
    logits = model(x, True)
    #print(y.shape)
    #print(logits.shape)
    #print(y)
    loss = loss_fn(logits, y.view(-1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(loss)

  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([64, 256, 32])
torch.Size([64, 256, 32])


RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [64, 32] but got: [64, 256].

In [16]:
#starts with a new line
with torch.inference_mode():
    sample = model.generator(torch.tensor([[0]]), 256)
    sample_tolist = sample[0].tolist()
    decoded = decode(sample_tolist)
print(decoded)


I peringnthe, pees.
KEETHeshen te pracours msthe be t ndlos bur manoulle his:

MENowisecht. l'ds u r gin, om.
dicowireabret, u geake llof Ano Vodsowhes thestourday Tofuto, f s oounghiackn.
Whe d ous NTh ING s'd wed t s I:
TE:
Yonat:
Ansid
Buropainth wey ic
